In [ ]:
url = "https://shop.vanger.com.tw/men/vanger_newcollections/va249burgundy"
url = "https://shop.vanger.com.tw/va249tan"

In [ ]:
import requests
from pyquery import PyQuery

In [ ]:
res = requests.get(url)

In [ ]:
res

In [ ]:
S = PyQuery(res.text)

In [ ]:
S("*[itemprop='offers']")

In [ ]:
res.text.split('''console.log("cloudmaker_stock(''')[-1].split(")")[0]

In [ ]:
product_sku = res.text.split('''console.log("cloudmaker_stock(''')[-1].split(")")[0]

In [ ]:
S("*[itemprop='offers'] h1").text()

In [ ]:
S('*[itemprop="offers"] *[itemprop="description"] p')

In [ ]:
"\n".join([PyQuery(p).text() for p in S('*[itemprop="offers"] *[itemprop="description"] p') if len(PyQuery(p)("a")) == 0])

In [ ]:
colors = [PyQuery(p)("a") for p in S('*[itemprop="offers"] *[itemprop="description"] p') if len(PyQuery(p)("a")) > 0][0]

In [ ]:
colors

In [ ]:
colors.map(lambda i,e:{
    "product_url":"https:" + PyQuery(e).attr("href"),
    "color": PyQuery(e)("img").attr("title"),
    "color_img_url": "https:" + PyQuery(e)("img").attr("src"),
})

In [ ]:
product_sku

In [ ]:
url = "https://shop.vanger.com.tw/index.php"

In [ ]:
res = requests.get(url,params={"route":"cloudmaker/order/wareHouseOrdernew",
                               "product_sku":product_sku})

In [ ]:
res

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(res.json())

In [ ]:
def exp_df(row):
    stores = row["cred_response"]["stock"][0]
    #df = pd.DataFrame([{"store":s,"qty":stores[s],"size":row["cred_response"]["size"]} for s in stores])
    return [{"store":s,"qty":stores[s],"size":row["cred_response"]["size"]} for s in stores]

store_inventory_df = pd.concat(list(map(pd.DataFrame,df.apply(exp_df,axis=1).tolist())))

In [ ]:
store_inventory_df["product_sku"] = product_sku

In [ ]:
store_inventory_df

In [ ]:
S(".product-table.europ-size").outer_html()

In [ ]:
S(".img-wrap").outer_html()

In [ ]:
    
S("*[type='hidden']").map(lambda i,e: (PyQuery(e).attr("id"), PyQuery(e).attr("value")))

In [ ]:

S("*[type='hidden']#product_id").attr("value")

In [ ]:
prod_id = S("*[type='hidden']#product_id").attr("value")

In [ ]:
url = "https://shop.vanger.com.tw/index.php?route=product/product_options&key={prod_id}&type=prod".format(prod_id = prod_id)

In [ ]:
res = requests.get(url)

In [ ]:
res

In [ ]:
SS = PyQuery(res.text)

In [ ]:
SS("option").map(lambda i,e:{
    "maybe_size_id": PyQuery(e).attr("value"),
    "size": PyQuery(e).text().replace("\xa0(暫無現貨)",""),
    "disable": PyQuery(e).text().endswith("\xa0(暫無現貨)")
})

In [ ]:
S('*[itemprop="price"]').attr("content")

In [ ]:
S(".discount").map(lambda i,e:PyQuery(e).outer_html())

In [ ]:
S(".tags a").map(lambda i,e:{
    "tag_name": PyQuery(e).text(),
    "tag_url": PyQuery(e).attr("href"),
})

In [ ]:
S(".description").text().replace("規格型號 ","")

In [ ]:
data = {
    "product_id": prod_id,
    "product_sku": product_sku,
    "price": S('*[itemprop="price"]').attr("content"),
    "discount_rule":S(".discount").map(lambda i,e:PyQuery(e).outer_html()),
    "tags": S(".tags a").map(lambda i,e:{
        "tag_name": PyQuery(e).text(),
        "tag_url": PyQuery(e).attr("href"),
    }),
    "sizes": SS("option").map(lambda i,e:{
        "maybe_size_id": PyQuery(e).attr("value"),
        "size": PyQuery(e).text().replace("\xa0(暫無現貨)",""),
        "disable": PyQuery(e).text().endswith("\xa0(暫無現貨)"),
    }),
    "image_wrap": S(".img-wrap").outer_html(),
    "sizing_info": S(".product-table.europ-size").outer_html(),
    "colors": colors.map(lambda i,e:{
        "product_url":"https:" + PyQuery(e).attr("href"),
        "color": PyQuery(e)("img").attr("title"),
        "color_img_url": "https:" + PyQuery(e)("img").attr("src"),
    }),
    "store_inventory": store_inventory_df.to_dict("records"),
    "spec_no":S(".description").text().replace("規格型號 ",""),
    
}

data